### 1. THƯ VIỆN

In [1]:
import pyodbc
import pandas as pd
import re
from pyvi import ViTokenizer

### 2. Tạo View bằng code Python

In [ ]:
conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};''SERVER=GIAHANHUYNH;''DATABASE=THEGIOIDIDONG;''Trusted_Connection=yes')

cur = conn.cursor()

create_view_query = """
CREATE VIEW dbo.DeviceView AS
SELECT 
    product_name, 
    current_price, 
    user_name, 
    user_rating, 
    user_comment, 
    user_time_comment
FROM Device;
"""
cur.execute(create_view_query)
conn.commit()
cur.close()
conn.close()

### 3. Đưa View vào dataframe

In [2]:
conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};''SERVER=GIAHANHUYNH;''DATABASE=THEGIOIDIDONG;''Trusted_Connection=yes')

cur = conn.cursor()
select_view_query = "SELECT * FROM dbo.DeviceView"
cur.execute(select_view_query)
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
cur.close()
conn.close()

df = pd.DataFrame.from_records(rows, columns=columns)
display(df.head())

,product_name,current_price,user_name,user_rating,user_comment,user_time_comment
0,Laptop Acer Aspire 3 A315 510P 32EF i3 N305/8G...,1.617354e+07,huyền trang,3,em muốn cài sang tiếng việt mà máy hong hỗ trợ,30
1,Laptop Acer Aspire 3 A315 510P 32EF i3 N305/8G...,1.617354e+07,Trịnh Thanh Tú,4,Em có mua máy này ở thế giới di động Sóc Trăng...,60
2,Laptop Acer Aspire 3 A315 510P 32EF i3 N305/8G...,1.617354e+07,NGUYỄN THỊ THANH NHÀN,5,"Mình chỉ sử dụng mục đích học tập, văn phòng n...",1
3,Laptop Acer Aspire 3 A315 510P 32EF i3 N305/8G...,1.617354e+07,bùi tiên,3,tôi không bật được micro trên laptop. xin được...,7
4,Laptop HP Gaming VICTUS 15 fa1139TX i5 12450H/...,1.849000e+07,Nguyễn Chí Vỹ,4,"Lap dùng ổn khi chơi game, làm tác vụ văn phòn...",7


### 4. Xử lý word

In [3]:
# Hàm VnCoreNLP
def VnCoreNLP(text):
    # Phân tách từ
    tokens = ViTokenizer.tokenize(text).split()
    # Kết nối các từ với nhau thành chuỗi
    p = ' '.join(tokens)
    # Loại bỏ khoảng trắng không cần thiết trước các dấu câu
    p = re.sub(r'\s([.,!?;{}()*/])', r'\1', p)
    return p


In [4]:
df['Processed_Comment'] = df['user_comment'].apply(VnCoreNLP)

display(df[['user_comment', 'Processed_Comment']])

,user_comment,Processed_Comment
0,em muốn cài sang tiếng việt mà máy hong hỗ trợ,em muốn cài sang tiếng việt mà máy hong hỗ_trợ
1,Em có mua máy này ở thế giới di động Sóc Trăng...,Em có mua máy này ở thế_giới di_động Sóc_Trăng...
2,"Mình chỉ sử dụng mục đích học tập, văn phòng n...","Mình chỉ sử_dụng mục_đích học_tập, văn_phòng n..."
3,tôi không bật được micro trên laptop. xin được...,tôi không bật được micro trên laptop. xin được...
4,"Lap dùng ổn khi chơi game, làm tác vụ văn phòn...","Lap dùng ổn khi chơi game, làm tác vụ văn_phòn..."
...,...,...
5227,Mình mới mua trả góp máy về tối hôm qua ngày 1...,Mình mới mua trả_góp máy về tối hôm_qua_ngày 1...
5228,Trải nghiệm sau 2 ngày ổn áp tầm giá này không...,Trải nghiệm sau 2 ngày ổn_áp tầm giá này không...
5229,Máy tệ hơn kỳ vọng,Máy_tệ hơn kỳ_vọng
5230,"Mình mua màu xanh,pin khỏe,dùng mượt . Mình cũ...","Mình mua màu xanh, pin_khỏe, dùng mượt. Mình c..."


### 5. Tạo Table và View mới trong DBMS

In [ ]:

conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};''SERVER=GIAHANHUYNH;''DATABASE=THEGIOIDIDONG;''Trusted_Connection=yes')

cur = conn.cursor()
create_table_query = """
CREATE TABLE dbo.ProcessedDevice (
    product_name NVARCHAR(255),
    current_price FLOAT,
    user_name NVARCHAR(255),
    user_rating NVARCHAR(MAX),
    user_comment NVARCHAR(MAX),
    processed_comment NVARCHAR(MAX),
    user_time_comment NVARCHAR(MAX)
);
"""
cur.execute(create_table_query)
conn.commit()
insert_query = """
INSERT INTO dbo.ProcessedDevice (product_name, current_price, user_name, user_rating, user_comment, processed_comment, user_time_comment)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
for index, row in df.iterrows():
    cur.execute(insert_query, 
                row['product_name'], 
                row['current_price'], 
                row['user_name'], 
                row['user_rating'], 
                row['user_comment'], 
                row['Processed_Comment'], 
                row['user_time_comment'])
conn.commit()
create_view_query = """
CREATE VIEW dbo.DeviceProcessedView AS
SELECT 
    product_name, 
    current_price, 
    user_name, 
    user_rating, 
    user_comment, 
    processed_comment, 
    user_time_comment
FROM dbo.ProcessedDevice;
"""
cur.execute(create_view_query)
conn.commit()
cur.close()
conn.close()

print("View DeviceProcessedView đã được tạo và lưu trữ thành công trong DBMS.")
